 # Comparison of GFlowNets with MCMC methods in 2D for the 4-simplex.

 

In [101]:
import numpy as np
import pandas as pd
import time

In [238]:
# cdf of a normal distribution. 
def cdfNormaDistribution(var, mean=0., deviation=1., dataPoints=1e5):
    """
    Calculate the cdf of a normal distribution. 
    
    Parameters
    ----------

    var: (float)
                - The point where we want ot find the cdf value.
    
    mean: (float)
                - The location of the normal distribution.

    deviation: (float)
                - The deviation of the noirmal distribution.

    dataPoints: (integer)
                - Number of points drawn from the normal distribution.
                - Number of points returned.

    cdfAccuracy: (integer)
                - Number of points to regard in the cdf.

    Return:
        -------
        cdf:
                - The cdf of the normal distribution.
                      
    """
    
    data = deviation*np.random.randn(int(dataPoints)) + mean

    # Sort the data in an ascending order.
    x = np.sort(data)

    # Get the values of the y-axis.
    y = np.arange(dataPoints)/ float(dataPoints)

    
    # Find the cumulative of the variable in question.
    cdf = y[np.argmax(x > var)]


    return cdf

In [244]:
def discreteNormalDistribution( dataPoints, mean=0.0, deviation=1.0):
    """
    Calculate the noraml disctribution of discrete variables. 
    
    Parameters
    ----------

    dataPoints: (integer)
                - Number of points drawn from the normal distribution.
                - Number of points returned.


    mean: (float)
                - The location of the normal distribution.

    deviation: (float)
                - The deviation of the noirmal distribution.

    Return:
        -------
        truncatedCoefficients: (array)
                 - The probability density function of a truncated discrete normal distribution.
                 - Its number is given by the dataPoints. 
                      
    """
    truncatedCoefficients = np.zeros(dataPoints, dtype=np.float64)

    for i in range(0,dataPoints):
        cdfDifferences = 0.0
        for n in range(-i, dataPoints-i):
            # Probability debsity function of a normal distribution. 
            cdfDifferences += (cdfNormaDistribution(n + 0.5, mean, deviation) - cdfNormaDistribution(n - 0.5, mean, deviation))
        # TODO: Should't the result be something like: (cdfNormaDistribution(i,deviation) - cdfNormaDistribution(i,deviation)) / cdfDifferences?
        truncatedCoefficients[i] = cdfDifferences
        
    return truncatedCoefficients

In [249]:
def VertexMCMC(spinJ, iterationsNumber, batchSize, deviation, verbosity):

    gridLength = 2*np.int64(spinJ) + 1


    if(iterationsNumber % batchSize != 0):
        raise ValueError("Number of iterations must be a multiple of batch size.")
    
    draws = np.empty([6,0], dtype=np.int64)
    truncatedCoefficients = discreteNormalDistribution(gridLength, 0, deviation)

    print("1")

    if(verbosity > 1):
        print("Truncated coefficients for j =",spinJ,"are",truncatedCoefficients)

    draw = np.empty(6, dtype=np.int64)
    gaussianDraw = np.empty(5, np.int64)

    amplitude = np.zeros(5, dtype=np.float64)
  
    while( np.any(amplitude[0:4] == 0 ) ):
        for i in range(1,5):
            draw[i] = np.random.randint(1, gridLength)
        amplitude = draw[:-1]

    print(amplitude)
    draw[-1] = 1 # Initial multiplicity

    if (verbosity > 1):
        print("Initial draw is", draw[1:5],"with amplitude", amplitude)

    # Proposed draw
    proposedDraw = np.empty(5, dtype=np.int64)

    print("2")


    acceptanceRatio = 0
    multiplicity = 1    # initial multiplicity counter
    drawFloatSample = 0
    
    RWMonitor = True

    dfAllBatches = pd.DataFrame([[], []], ["acceptance rate (%)", "run time (s)"])

    batchCounter = 0

    print(draws, "\t", draw)

    for n in range(2, iterationsNumber+1):
        if(verbosity > 1):
            print("Iteration $(n)---------------------------------------------------------------")

        timeInitial = time.time_ns # Initial time

        print("3")

        if( n % batchSize ==0 ):
            if(verbosity >1):
                print(n,"iterations reached, tome to store",batchSize,"draws.")
            np.append(draws[:-1,], draw[:-1] - 1)
            np.append(draws[-1], multiplicity)

            if(verbosity > 1):
                print("The last draw", draw[:-1],"has been stored with multiplicity", draw[-1])

    # return amplitude


    


In [251]:
VertexMCMC(1000, 2, 2, 1, 2)

KeyboardInterrupt: 

In [ ]:
function VertexMCMC(
    b::Int64,
    draws_folder::String,
    verbosity::Int64,
)
    draws = ElasticArray{Int64}(undef, 6, 0)

    for n = 1:1:N
     
            if (verbosity > 1)
                println(
                    "The last draw $(draw[1:5]) has been stored with multiplicity $(draw[6])\n",
                )
            end

            acceptance_percentage = acceptance_ratio * 100 / batch_size

            println(
                "$(acceptance_percentage)% of proposed draws have been accepted in this batch (in master chain)\n",
            )

            batch_counter += 1

            CSV.write(
                "$(draws_folder)/draws_batch_n_$(batch_counter).csv",
                DataFrame(
                    transpose(draws),
                    [
                        "intertwiner 1",
                        "intertwiner 2",
                        "intertwiner 3",
                        "intertwiner 4",
                        "intertwiner 5",
                        "multiplicity",
                    ],
                ),
            )

            T_fin = time_ns()

            push!(
                df_all_batches,
                [acceptance_percentage, round((T_fin - T_in) * 10^(-9), digits = 4)],
            )

            acceptance_ratio = 0
            multiplicity = 1
            resize!(draws, 6, 0)

            if (n == N)
                CSV.write("$(draws_folder)/statistics_batches.csv", df_all_batches)
            end

            continue

        end

        RW_monitor = true

        # Sampling proposed draw   
        Cx = Cx_prop = 1.0
        for i = 1:1:5
            while true
                rand!(Normal_distribution, draw_float_sample)
                @inbounds gaussian_draw[i] = round(Int64, draw_float_sample[1])
                @inbounds prop_draw[i] = draw[i] + gaussian_draw[i]
                @inbounds !(1 <= prop_draw[i] && prop_draw[i] <= (D)) || break
            end
            if (gaussian_draw[i] != 0)
                RW_monitor = false
            end
            @inbounds Cx *= Truncated_Coefficients[draw[i]]
            @inbounds Cx_prop *= Truncated_Coefficients[prop_draw[i]]
        end

        if (RW_monitor == true)

            if (verbosity > 1)
                println(
                    "The prop_draw below:\n$(prop_draw[1:5])\nturns out to be equal to the current draw:\n$(draw[1:5])\nso that the multiplicity of the current draw is raised to $(multiplicity + 1)\n",
                )
            end

            acceptance_ratio += 1
            multiplicity += 1

            continue

        else

            if (verbosity > 1)
                println(
                    "draw is:\n$(draw[1:5])\nprop_draw is:\n$(prop_draw[1:5])\namp is $(amp)\n",
                )
            end


            Prop_amp =
                A[prop_draw[1], prop_draw[2], prop_draw[3], prop_draw[4], prop_draw[5]]

            p = min(1.0, (((Prop_amp^2) / (amp^2))) * (Cx / Cx_prop))

            if (isnan(p))
                error(
                    "got NaN while computing densities ratio: prop_draw = $(prop_draw), amp = $(amp)",
                )
            end

            r = rand()

            if (r < p)

                if (verbosity > 1)
                    println(
                        "prop_draw $(prop_draw) was accepted, since p=$(p) and r=$(r)\n",
                    )
                end

                if (n > b)

                    resize!(draws, 6, size(draws)[2] + 1)
                    #draw[6] = multiplicity
                    draws[1:5, end] .= draw[1:5] .- 1
                    draws[6, end] = multiplicity

                    if (verbosity > 1)
                        println(
                            "The old draw $(draw[1:5]) has been stored with multiplicity $(draw[6])\n",
                        )
                    end

                end

                multiplicity = 1

                for i = 1:5
                    draw[i] = prop_draw[i]
                end

                amp = Prop_amp
                acceptance_ratio += 1

                if (verbosity > 1)
                    println("Now the new draw is $(draw[1:5])\nthe new amp is $(amp)\n")
                end
            else
                multiplicity += 1
                if (verbosity > 1)
                    println(
                        "prop_draw $(prop_draw) was rejected, since p=$(p) and r=$(r)\nThe current draw $(draw[1:5]) remains the same and its multiplicity is $(multiplicity)\namp remains $(amp)\n",
                    )
                end
            end

        end

    end

end